In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from imblearn.combine import SMOTETomek
from imblearn.over_sampling import ADASYN, BorderlineSMOTE, RandomOverSampler, SMOTE
from imblearn.under_sampling import NearMiss, RandomUnderSampler, TomekLinks
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix, matthews_corrcoef, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [18]:
full_dataset = pd.read_csv('/content/drive/MyDrive/ABM/out_pattern_all.csv', sep=';')
full_dataset = full_dataset.drop_duplicates()
# full_dataset = full_dataset.drop('Column1', axis=1)
full_dataset

,noise_traders,noise_traders_cash,noise_traders_wealth,market_maker_cash,market_maker_wealth,mm_type1,mm_type2,mm_type3,mm_spread_width_type1,mm_spread_width_type2,...,long_term_cash,long_term_wealth,high_freq,high_freq_cash,high_freq_wealth,high_freq_lag,high_freq_threshold,shock_step,shock_price_change,res
0,100,1000,30,50000,2000,1,3,3,0.005,0.03,...,3000,50,150,5000,100,3,0.02,500,0.9,7
1,100,3000,30,200000,2000,1,1,1,0.150,0.03,...,1000,50,150,5000,50,7,0.01,700,0.9,3
2,200,1000,30,50000,1000,3,2,3,0.005,0.05,...,3000,100,150,3000,100,7,0.03,500,0.9,7
3,100,3000,30,50000,1000,1,3,1,0.005,0.03,...,3000,50,70,5000,70,7,0.02,700,0.5,7
4,300,2000,10,100000,1000,1,3,1,0.010,0.01,...,5000,50,100,3000,50,5,0.01,300,0.5,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,100,5000,10,200000,2000,2,1,2,0.010,0.01,...,1000,100,70,1000,100,3,0.01,500,0.5,7
2996,100,3000,20,200000,1000,1,1,3,0.005,0.05,...,5000,30,70,5000,50,7,0.01,700,0.5,7
2997,100,3000,20,50000,2000,1,3,1,0.150,0.01,...,3000,30,70,3000,70,7,0.03,300,0.8,8
2998,200,3000,30,50000,3000,3,1,1,0.010,0.05,...,1000,50,100,5000,100,7,0.01,500,0.9,3


In [19]:
X = full_dataset.drop('res', axis=1)
y = full_dataset.res
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.head()
y_train.head()

835     7
1674    7
519     7
913     9
1444    2
Name: res, dtype: int64

In [20]:
def compute_metrics(y_test, y_pred):
    print('Accuracy: {:.5f}'.format(accuracy_score(y_test, y_pred)))
    print('F-score: {:.5f}'.format(f1_score(y_test, y_pred, average="weighted")))
    print('Precision: {:.5f}'.format(precision_score(y_test, y_pred, average="weighted")))
    print('Recall: {:.5f}'.format(recall_score(y_test, y_pred, average="weighted")))
    print('Accuracy (balanced): {:.5f}'.format(balanced_accuracy_score(y_test, y_pred)))
    print('MCC: {:.5f}'.format(matthews_corrcoef(y_test, y_pred)))

def compute_confusion_matrix(y_test, y_pred):
    compute_metrics(y_test, y_pred)
    return pd.DataFrame(
        confusion_matrix(y_test, y_pred, labels=[1, 2, 3, 4, 5, 6, 7, 8, 9]),
        columns=['a(x) = 1', 'a(x) = 2', 'a(x) = 3', 'a(x) = 4', 'a(x) = 5', 'a(x) = 6', 'a(x) = 7', 'a(x) = 8', 'a(x) = 9'],
        index=['y = 1', 'y = 2', 'y = 3', 'y = 4', 'y = 5', 'y = 6', 'y = 7', 'y = 8', 'y = 9'],
    ).T

In [21]:
num_trans = StandardScaler()
numeric = [c for c in X_train.columns]

column_transformer = ColumnTransformer([
 ('all_num', num_trans, numeric)],
remainder='passthrough'
)

X_train = column_transformer.fit_transform(X_train)

In [27]:
X = full_dataset.drop('res', axis=1)
y = full_dataset.res
PredictorScaler = StandardScaler()
PredictorScalerFit = PredictorScaler.fit(X)
X=PredictorScalerFit.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train


array([[-0.00445857, -1.18592479, -1.20607461, ...,  0.0036937 ,
        -1.20656963, -1.40505219],
       [-0.00445857,  0.16735569,  1.22961966, ..., -1.22754056,
        -1.20656963, -1.40505219],
       [-1.22043154, -0.50928455,  0.01177252, ...,  1.23492797,
         0.01218757,  0.96701115],
       ...,
       [-1.22043154,  1.52063616,  0.01177252, ...,  0.0036937 ,
         0.01218757, -1.40505219],
       [ 1.2115144 ,  0.16735569, -1.20607461, ...,  1.23492797,
         0.01218757,  0.37399532],
       [ 1.2115144 , -0.50928455,  1.22961966, ...,  1.23492797,
         0.01218757,  0.96701115]])

In [28]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(X_train, y_train)
gnb_predictions = gnb.predict(X_test)
 
f1_scNB = f1_score(y_test, gnb_predictions, average='weighted')
print(f1_scNB)
 
# creating a confusion matrix
cm = confusion_matrix(y_test, gnb_predictions)
print(f1_score(y_test, y_pred, average='weighted'))
compute_confusion_matrix(y_test, gnb_predictions)

0.6267652872362917
0.30893901420217207
Accuracy: 0.66111
F-score: 0.62677
Precision: 0.63807
Recall: 0.66111
Accuracy (balanced): 0.36396
MCC: 0.49434


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,y = 1,y = 2,y = 3,y = 4,y = 5,y = 6,y = 7,y = 8,y = 9
a(x) = 1,0,0,0,0,0,0,0,0,0
a(x) = 2,0,4,0,0,0,0,0,0,6
a(x) = 3,0,0,91,0,0,0,1,24,0
a(x) = 4,0,0,1,3,0,0,1,1,0
a(x) = 5,0,0,0,0,1,0,0,0,0
a(x) = 6,1,0,0,0,0,20,8,3,0
a(x) = 7,1,11,16,3,5,20,395,89,39
a(x) = 8,0,0,21,0,0,0,20,43,0
a(x) = 9,0,15,0,12,2,0,5,0,38


In [62]:
param_grid = {
    'max_depth': [5, 8, 9, 10, 12, 15],
    'n_estimators': [100, 300, 200, 500]
}

model = GradientBoostingClassifier()

pipe_cv = GridSearchCV(model, param_grid, n_jobs=-1, cv=5, scoring='f1_samples')

pipe_cv.fit(X_train, y_train)
print(pipe_cv.best_estimator_)
print(pipe_cv.best_score_)
y_pred = pipe_cv.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
  category=UserWarning,


GradientBoostingClassifier(max_depth=5)
nan


In [33]:
f1_scNB = f1_score(y_test, y_pred, average='macro')
print(f1_scNB)
print(f1_score(y_test, y_pred, average='weighted'))
compute_confusion_matrix(y_test, y_pred)

0.452404945058258
0.7122583403433388
Accuracy: 0.73333
F-score: 0.71226
Precision: 0.72011
Recall: 0.73333
Accuracy (balanced): 0.42512
MCC: 0.61106


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,y = 1,y = 2,y = 3,y = 4,y = 5,y = 6,y = 7,y = 8,y = 9
a(x) = 1,0,0,0,0,0,0,0,0,0
a(x) = 2,0,8,0,1,0,0,0,0,2
a(x) = 3,0,0,108,0,0,1,8,30,0
a(x) = 4,0,0,1,3,0,0,0,0,0
a(x) = 5,0,0,0,0,0,0,0,1,0
a(x) = 6,0,0,0,0,0,21,4,1,1
a(x) = 7,2,5,4,2,4,16,392,55,24
a(x) = 8,0,0,16,0,2,1,20,73,1
a(x) = 9,0,17,0,12,2,1,6,0,55


In [34]:
# OK
X = full_dataset.drop('res', axis=1)
y = full_dataset.res
PredictorScaler = StandardScaler()
PredictorScalerFit = PredictorScaler.fit(X)
X=PredictorScalerFit.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

model = RandomForestClassifier()

param_grid = {
    'max_depth': [12, 15, 17],
    'n_estimators': [100, 150, 200, 250]
}

pipe_cv = GridSearchCV(model, param_grid, n_jobs=-1, cv=5, scoring='f1_weighted')
pipe_cv.fit(X_train, y_train)
print(pipe_cv.best_estimator_)
print(pipe_cv.best_score_)
y_pred = pipe_cv.predict(X_test)

f1_score(y_test, y_pred, average='weighted')
compute_confusion_matrix(y_test, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  UserWarning,


RandomForestClassifier(max_depth=15)
0.6283506659161294
Accuracy: 0.70667
F-score: 0.65551
Precision: 0.68305
Recall: 0.70667
Accuracy (balanced): 0.31352
MCC: 0.52720


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,y = 1,y = 2,y = 3,y = 4,y = 5,y = 6,y = 7,y = 8,y = 9
a(x) = 1,0,0,0,0,0,0,0,0,0
a(x) = 2,0,0,0,0,0,0,0,0,0
a(x) = 3,0,0,76,0,0,1,4,19,0
a(x) = 4,0,0,0,0,0,0,0,0,0
a(x) = 5,0,0,0,0,0,0,0,0,0
a(x) = 6,0,0,0,0,0,1,0,0,0
a(x) = 7,0,7,30,8,2,23,385,63,27
a(x) = 8,0,0,11,0,0,0,5,37,0
a(x) = 9,0,8,0,10,0,1,1,0,31


In [12]:
# OK
X = full_dataset.drop('res', axis=1)
y = full_dataset.res
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = RandomForestClassifier()

param_grid = {
    'max_depth': [11, 12, 13, 14],
    'n_estimators': [180, 190, 200, 210, 230]
}

pipe_cv = GridSearchCV(model, param_grid, n_jobs=-1, cv=5, scoring='f1_weighted')
pipe_cv.fit(X_train, y_train)
print(pipe_cv.best_estimator_)
print(pipe_cv.best_score_)
y_pred = pipe_cv.predict(X_test)

f1_score(y_test, y_pred, average='weighted')
compute_confusion_matrix(y_test, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


RandomForestClassifier(max_depth=13, n_estimators=190)
0.6253905861493935
Accuracy: 0.70444
F-score: 0.26647
Precision: 0.33002
Recall: 0.25338
Accuracy (balanced): 0.25338
MCC: 0.51840


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,y = 1,y = 2,y = 3,y = 4,y = 5,y = 6,y = 7,y = 8,y = 9
a(x) = 1,0,0,0,0,0,0,0,0,0
a(x) = 2,0,0,0,0,0,0,0,0,0
a(x) = 3,0,0,108,0,0,0,6,21,0
a(x) = 4,0,0,0,0,0,0,0,0,0
a(x) = 5,0,0,0,0,0,0,0,0,0
a(x) = 6,0,0,0,1,0,0,0,0,0
a(x) = 7,1,12,25,5,7,26,467,91,46
a(x) = 8,0,0,16,0,0,2,3,36,0
a(x) = 9,0,1,0,2,0,0,1,0,23


In [5]:
X = full_dataset.drop('res', axis=1)
y = full_dataset.res
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.head()

,noise_traders,noise_traders_cash,noise_traders_wealth,market_maker_cash,market_maker_wealth,mm_type1,mm_type2,mm_type3,mm_spread_width_type1,mm_spread_width_type2,...,long_term_type3,long_term_cash,long_term_wealth,high_freq,high_freq_cash,high_freq_wealth,high_freq_lag,high_freq_threshold,shock_step,shock_price_change
879,300,1000,10,200000,1000,3,3,3,0.005,0.05,...,100,1000,50,150,3000,50,3,0.02,300,0.5
815,100,3000,20,100000,3000,1,2,1,0.150,0.05,...,50,3000,100,70,3000,100,5,0.02,500,0.8
832,200,3000,10,100000,2000,1,2,1,0.005,0.05,...,100,1000,30,150,5000,50,3,0.03,500,0.9
890,200,5000,20,50000,2000,1,3,1,0.005,0.03,...,100,5000,50,70,1000,100,7,0.01,700,0.5
189,300,5000,20,200000,2000,1,2,3,0.005,0.05,...,100,3000,100,100,5000,100,3,0.01,300,0.8


In [35]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 82 kB/s 


In [36]:
# CATBOOST
from catboost import CatBoostClassifier

X = full_dataset.drop('res', axis=1)
y = full_dataset.res
PredictorScaler = StandardScaler()
PredictorScalerFit = PredictorScaler.fit(X)
X=PredictorScalerFit.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = CatBoostClassifier(iterations=90)

model.fit(X_train, y_train, plot=True)
predictions = model.predict(X_test)

compute_confusion_matrix(y_test, predictions)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.5
0:	learn: 1.2622880	total: 51.4ms	remaining: 4.58s
1:	learn: 1.1350795	total: 56.6ms	remaining: 2.49s
2:	learn: 0.9782597	total: 61.7ms	remaining: 1.79s
3:	learn: 0.9190566	total: 66.9ms	remaining: 1.44s
4:	learn: 0.8639230	total: 72.1ms	remaining: 1.23s
5:	learn: 0.8273109	total: 77.3ms	remaining: 1.08s
6:	learn: 0.7767090	total: 82.2ms	remaining: 974ms
7:	learn: 0.7360580	total: 87.4ms	remaining: 896ms
8:	learn: 0.7085393	total: 92.7ms	remaining: 834ms
9:	learn: 0.6564904	total: 98.2ms	remaining: 786ms
10:	learn: 0.6283942	total: 104ms	remaining: 743ms
11:	learn: 0.6012411	total: 113ms	remaining: 737ms
12:	learn: 0.5745330	total: 119ms	remaining: 704ms
13:	learn: 0.5389759	total: 124ms	remaining: 673ms
14:	learn: 0.5181399	total: 129ms	remaining: 645ms
15:	learn: 0.4985937	total: 142ms	remaining: 659ms
16:	learn: 0.4793383	total: 156ms	remaining: 671ms
17:	learn: 0.4503173	total: 168ms	remaining: 671ms
18:	learn: 0.4321410	total: 172ms	remaining: 644ms
19:	le

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,y = 1,y = 2,y = 3,y = 4,y = 5,y = 6,y = 7,y = 8,y = 9
a(x) = 1,0,0,0,0,0,0,0,0,0
a(x) = 2,0,3,0,0,0,0,0,0,1
a(x) = 3,0,0,105,0,1,1,2,17,0
a(x) = 4,0,0,0,1,0,1,0,0,0
a(x) = 5,0,0,0,0,0,0,0,0,0
a(x) = 6,0,0,0,1,0,12,0,2,1
a(x) = 7,1,11,11,4,3,22,436,43,22
a(x) = 8,0,0,29,0,0,1,18,78,2
a(x) = 9,0,12,0,5,1,2,5,0,46


In [38]:
importance = model.get_feature_importance(prettified=True)

In [56]:
ccol_names = []
for i in importance['Feature Id']:
  col_names.append(full_dataset.columns[int(i)])
importance['Parameter_name'] = col_names
importance

,Feature Id,Importances,column,Parameter_name
0,12,21.790157,mm_hard_line_sell,mm_hard_line_sell
1,13,16.217727,mm_hard_line_buy,mm_hard_line_buy
2,0,8.431319,noise_traders,noise_traders
3,1,5.174231,noise_traders_cash,noise_traders_cash
4,8,4.887022,mm_spread_width_type1,mm_spread_width_type1
5,2,4.343135,noise_traders_wealth,noise_traders_wealth
6,7,3.652275,mm_type3,mm_type3
7,6,3.346988,mm_type2,mm_type2
8,10,3.300242,mm_spread_width_type3,mm_spread_width_type3
9,9,2.881963,mm_spread_width_type2,mm_spread_width_type2


In [61]:
full_dataset.res.value_counts()


7    1542
8     504
3     454
9     241
6     108
2      75
4      51
5      23
1       2
Name: res, dtype: int64